In [2]:
class GoalBasedAgentDFS:
    def __init__(self, goal):
        self.goal = goal

    def formulate_goal(self, percept):
        return "goal reached" if percept == self.goal else "searching"

    def act(self, percept, environment):
        goal_status = self.formulate_goal(percept)
        if goal_status == "goal reached":
            return f"goal {self.goal} found!"
        return environment.dfs_search(percept, self.goal)

class EnvironmentDFS:
    def __init__(self, graph):
        self.graph = graph

    def get_percept(self, node):
        return node

    def dfs_search(self, start, goal):
        visited = []
        stack = [start]

        while stack:
            node = stack.pop()  #LIFO
            print(f"visiting {node}")

            if node == goal:
                return f"goal {goal} found!"

            for neighbor in reversed(self.graph.get(node, [])):  #reverse to maintain correct DFS order
                if neighbor not in visited:
                    visited.append(neighbor)
                    stack.append(neighbor)
                    
        return "goal not found"

def run_dfs_agent(start_node, goal_node, graph):
    agent = GoalBasedAgentDFS(goal_node)
    environment = EnvironmentDFS(graph)
    percept = environment.get_percept(start_node)
    print(agent.act(percept, environment))

graph = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F', 'G'],
    'D': ['H'],
    'E': [],
    'F': ['I'],
    'G': [],
    'H': [],
    'I': []
}

run_dfs_agent('A', 'I', graph)

visiting A
visiting B
visiting D
visiting H
visiting E
visiting C
visiting F
visiting I
goal I found!


### Depth Limited Search (DLS)

In [3]:
class GoalBasedAgentDLS:
    def __init__(self, goal, depth_limit):
        self.goal = goal
        self.depth_limit = depth_limit

    def formulate_goal(self, percept):
        return "goal reached" if percept == self.goal else "searching"

    def act(self, percept, environment):
        goal_status = self.formulate_goal(percept)
        if goal_status == "goal reached":
            return f"goal {self.goal} found!"
        return environment.dls_search(percept, self.goal, self.depth_limit)

class EnvironmentDLS:
    def __init__(self, graph):
        self.graph = graph

    def get_percept(self, node):
        return node

    def dls_search(self, start, goal, limit, depth=0):
        if start == goal:
            return f"goal {goal} found!"

        if depth >= limit:
            return "depth limit reached!"

        print(f"visiting {start} at depth {depth}")

        for neighbor in self.graph.get(start, []):
            result = self.dls_search(neighbor, goal, limit, depth + 1)
            if result.startswith("goal"):
                return result

        return "goal not found"

def run_dls_agent(start_node, goal_node, graph, depth_limit):
    agent = GoalBasedAgentDLS(goal_node, depth_limit)
    environment = EnvironmentDLS(graph)
    percept = environment.get_percept(start_node)
    print(agent.act(percept, environment))

graph = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F', 'G'],
    'D': ['H'],
    'E': [],
    'F': ['I'],
    'G': [],
    'H': [],
    'I': []
}

run_dls_agent('A', 'I', graph, 3)  #search depth limited to 3


visiting A at depth 0
visiting B at depth 1
visiting D at depth 2
goal not found


### Uniform Cost Search (UCS)

In [4]:
class UtilityBasedAgentUCS:
    def __init__(self, goal):
        self.goal = goal

    def formulate_goal(self, percept):
        return "goal reached" if percept == self.goal else "searching"

    def act(self, percept, environment):
        goal_status = self.formulate_goal(percept)
        if goal_status == "goal reached":
            return f"goal {self.goal} found!"
        return environment.ucs_search(percept, self.goal)

class EnvironmentUCS:
    def __init__(self, graph):
        self.graph = graph

    def get_percept(self, node):
        return node

    def ucs_search(self, start, goal):
        #frontier list
        frontier = [(start, 0)]  #(node, cost)
        visited = set()  #visited nodes
        cost_so_far = {start: 0}  #stores the min cost to reach each node
        came_from = {start: None}  #for reconstruction of path

        while frontier:
            #sort frontier based on cost
            frontier.sort(key=lambda x: x[1])
            current_node, current_cost = frontier.pop(0)  #pop lowest cost node

            if current_node in visited:     #if alr visited, skip it.
                continue

            visited.add(current_node) #mark node as visited.

            #goal check: reconstruct path & then return
            if current_node == goal:
                path = []
                while current_node is not None:
                    path.append(current_node)
                    current_node = came_from[current_node]
                path.reverse()
                return f"goal found! path: {path}, total cost: {current_cost}"

            for neighbor, weight in self.graph.get(current_node, []):
                new_cost = current_cost + weight
                #update cost only if its lower than a previously found path
                if neighbor not in cost_so_far or new_cost < cost_so_far[neighbor]:
                    cost_so_far[neighbor] = new_cost
                    frontier.append((neighbor, new_cost))
                    came_from[neighbor] = current_node 

        return "goal not found"

def run_ucs_agent(start_node, goal_node, graph):
    agent = UtilityBasedAgentUCS(goal_node)
    environment = EnvironmentUCS(graph)
    percept = environment.get_percept(start_node)
    print(agent.act(percept, environment))

graph = {
    'A': [('B', 1), ('C', 4)],
    'B': [('D', 2), ('E', 5)],
    'C': [('F', 3), ('G', 6)],
    'D': [('H', 7)],
    'E': [],
    'F': [('I', 2)],
    'G': [],
    'H': [],
    'I': []
}

run_ucs_agent('A', 'I', graph)


goal found! path: ['A', 'C', 'F', 'I'], total cost: 9


### Q2

In [5]:
class TSPEnvironment:
    def __init__(self):
        self.distances = [
            [0, 10, 15, 20],  #city 1
            [10, 0, 35, 25],  #city 2
            [15, 35, 0, 30],  #city 3
            [20, 25, 30, 0]   #city 4
        ]
        self.num_cities = len(self.distances)

    def get_distance(self, city1, city2):
        return self.distances[city1][city2]


class TSPAgent:
    def __init__(self, environment):
        self.environment = environment
        self.best_cost = float('inf')
        self.best_route = []

    def dfs(self, city, visited, path, cost):
        """ Depth-First Search with Branch and Bound for TSP. """
        #if all cities visited, return to start
        if len(path) == self.environment.num_cities:
            total_cost = cost + self.environment.get_distance(city, 0)
            if total_cost < self.best_cost:
                self.best_cost = total_cost
                self.best_route = path[:] + [0]  #complete the cycle
            return

        #explore all possible next cities
        for next_city in range(self.environment.num_cities):
            if next_city not in visited and next_city != city:
                new_cost = cost + self.environment.get_distance(city, next_city)

                #branch & bound: prune if cost is already higher than best_cost
                if new_cost < self.best_cost:
                    visited.add(next_city)
                    self.dfs(next_city, visited, path + [next_city], new_cost)
                    visited.remove(next_city)

    def find_optimal_route(self):
        self.dfs(0, {0}, [0], 0)
        return [city + 1 for city in self.best_route], self.best_cost  


environment = TSPEnvironment()
agent = TSPAgent(environment)
optimal_route, min_cost = agent.find_optimal_route()
print(f"optimal route: {optimal_route}")
print(f"minimum cost: {min_cost}")


optimal route: [1, 2, 4, 3, 1]
minimum cost: 80


### Q3

In [7]:
class Graph:
    """graph class w/ adjacency list representation """
    def __init__(self):
        self.adj_list = {}

    def add_edge(self, u, v):
        if u not in self.adj_list:
            self.adj_list[u] = []
        self.adj_list[u].append(v)

    def get_neighbors(self, node):
        return self.adj_list.get(node, [])

#agent for iterative deepening DFS
class IDDFSAgent:
    def __init__(self, graph):
        self.graph = graph

    def DLS(self, node, goal, depth, visited):
        if node == goal:
            return [node]  
        if depth == 0:
            return None  #depth limit reached

        visited.add(node)
        for neighbor in self.graph.get_neighbors(node):
            if neighbor not in visited:
                path = self.DLS(neighbor, goal, depth - 1, visited)
                if path:
                    return [node] + path  #construct path
        visited.remove(node)
        return None  #no path found

    #implements IDDFS by gradually increasing depth limit
    def iterative_deepening_dfs(self, start, goal, max_depth):
        for depth in range(max_depth + 1):
            visited = set()
            path = self.DLS(start, goal, depth, visited)
            if path:
                return path  #return 1st found path
        return None  #no path was found within max depth


graph = Graph()
graph.add_edge(0, 1)
graph.add_edge(0, 2)
graph.add_edge(1, 3)
graph.add_edge(1, 4)
graph.add_edge(2, 5)
graph.add_edge(2, 6)

agent = IDDFSAgent(graph)

start_node, goal_node, max_depth = 0, 6, 3
path = agent.iterative_deepening_dfs(start_node, goal_node, max_depth)
print(f"IDDFS path from {start_node} to {goal_node}: {path}")


IDDFS path from 0 to 6: [0, 2, 6]


In [ ]:
from collections import deque

class BiDirectionalGraph:
    """graph class for bidirectional search"""
    def __init__(self):
        self.adj_list = {}

    def add_edge(self, u, v):
        if u not in self.adj_list:
            self.adj_list[u] = []
        if v not in self.adj_list:
            self.adj_list[v] = []
        self.adj_list[u].append(v)
        self.adj_list[v].append(u)

    def get_neighbors(self, node):
        return self.adj_list.get(node, [])

#agent for performing bidirectional search
class BidirectionalSearchAgent:
    def __init__(self, graph):
        self.graph = graph

    def bidirectional_search(self, start, goal):
        if start == goal:
            return [start]

        forward_queue = deque([start])
        backward_queue = deque([goal])

        forward_visited = {start: None}
        backward_visited = {goal: None}

        while forward_queue and backward_queue:
            #forward BFS step
            if self._bfs_step(forward_queue, forward_visited, backward_visited):
                return self._construct_path(forward_visited, backward_visited)

            #backward BFS step
            if self._bfs_step(backward_queue, backward_visited, forward_visited):
                return self._construct_path(forward_visited, backward_visited)

        return None 

    #performs one step of BFS
    def _bfs_step(self, queue, visited, other_visited):
        if queue:
            node = queue.popleft()
            for neighbor in self.graph.get_neighbors(node):
                if neighbor not in visited:
                    visited[neighbor] = node
                    queue.append(neighbor)

                if neighbor in other_visited:  #intersection found
                    return True
        return False

    #constructs the final path from start to goal
    def _construct_path(self, forward_visited, backward_visited):
        path = []
        meeting_point = list(set(forward_visited) & set(backward_visited))[0]

        #path built from start to meeting pt
        node = meeting_point
        while node is not None:
            path.append(node)
            node = forward_visited[node]
        path.reverse()

        #path built from meeting pt to goal
        node = backward_visited[meeting_point]
        while node is not None:
            path.append(node)
            node = backward_visited[node]
        return path

graph = BiDirectionalGraph()
graph.add_edge(0, 1)
graph.add_edge(0, 2)
graph.add_edge(1, 3)
graph.add_edge(1, 4)
graph.add_edge(2, 5)
graph.add_edge(2, 6)
graph.add_edge(4, 6)  #creates alt path

agent = BidirectionalSearchAgent(graph)

start_node, goal_node = 0, 6
path = agent.bidirectional_search(start_node, goal_node)
print(f"bidirectional search path from {start_node} to {goal_node}: {path}")


bidirectional search path from 0 to 6: [0, 2, 6]
